In [3]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

# Process the data and convert it into a pandas DataFrame
data = []
with open("processed_parallel.txt", "r") as file:
    for line in file:
        values = line.strip().split()
        frame = float(values[0])
        pedestrian_id = float(values[1])
        x = float(values[-4])
        y = float(values[-2])
        data.append([frame, pedestrian_id, x, y])

df = pd.DataFrame(data, columns=["frame", "pedestrian_id", "x", "y"])


In [4]:
df

,frame,pedestrian_id,x,y
0,0.0,1.0,5.577136,3.434354
1,0.0,2.0,5.637184,-1.759812
2,1.0,1.0,5.672341,3.290679
3,1.0,2.0,6.124671,-1.741960
4,2.0,1.0,5.727101,3.201846
...,...,...,...,...
77,38.0,2.0,6.832444,1.056036
78,39.0,1.0,6.901539,-0.384919
79,39.0,2.0,6.822493,1.056735
80,40.0,1.0,6.865525,0.180890


In [6]:

# Filter out rows with missing coordinates
df = df[(df["x"] != -1.0) & (df["y"] != -1.0)]

# Define the constant velocity model function
def constant_velocity_model(t, x0, y0, vx, vy):
    x = x0 + vx * t
    y = y0 + vy * t
    return np.column_stack((x, y))

# Get unique pedestrian IDs
pedestrian_ids = df["pedestrian_id"].unique()

# Iterate over each pedestrian and make predictions
for pedestrian_id in pedestrian_ids:
    pedestrian_df = df[df["pedestrian_id"] == pedestrian_id]
    
    # Extract time steps and positions for the current pedestrian
    time_steps = pedestrian_df["frame"].values
    positions = pedestrian_df[["x", "y"]].values
    
    # Fit the constant velocity model to the trajectory data
    initial_guess = [positions[0, 0], positions[0, 1], 1.0, 1.0]
    popt, _ = curve_fit(constant_velocity_model, time_steps, positions.ravel(), p0=initial_guess)
    
    # Extract the fitted parameters
    x0, y0, vx, vy = popt
    
    # Generate future time steps for prediction
    last_time_step = time_steps[-1]
    future_time_steps = np.arange(last_time_step + 1, last_time_step + 13)
    
    # Predict future positions using the constant velocity model
    predicted_positions = constant_velocity_model(future_time_steps, x0, y0, vx, vy)
    
    # Print the predicted trajectory for the current pedestrian
    print(f"Pedestrian ID: {pedestrian_id}")
    print("Predicted trajectory:")
    print(predicted_positions)
    print()

ValueError: operands could not be broadcast together with shapes (41,2) (82,) 

In [ ]:
import numpy as np
from scipy.interpolate import CubicSpline

# Create sample pedestrian trajectory data
trajectory_data = np.array([[0.0, 0.0], [1.0, 2.0], [2.0, 3.0], [3.0, 1.0], [4.0, 2.0]])

# Extract time steps and positions from the trajectory data
time_steps = trajectory_data[:, 0]
x_positions = trajectory_data[:, 1]

# Create a cubic spline interpolation function
spline = CubicSpline(time_steps, x_positions)

# Generate future time steps for prediction
num_predictions = 5
future_time_steps = np.arange(time_steps[-1] + 1, time_steps[-1] + num_predictions + 1)

# Predict future positions using the spline interpolation
predicted_positions = spline(future_time_steps)

# Print the predicted trajectory
print("Predicted trajectory:")
print(np.column_stack((future_time_steps, predicted_positions)))